In [2]:
import sys
import os
sys.path.append(os.path.abspath("D:\\Python\\commodity_futures_price"))

import math

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# %load_ext autoreload
# %autoreload 2
%reload_ext autoreload
from model.savvy_sh import SavvySh

In [3]:
df_dict_cc1 = {}
valid_dates = pd.date_range(start="2011-01-01", end="2024-12-31", freq="B")
# Read each file in below folder
path = "../data/raw/"
files = os.listdir(path)
for file in files:
    if file.endswith(".csv"):
        # Exclude LB, JN, LL, LN, LT, LX. due to data correctness issue
        if file.startswith(("LB", "JN", "LL", "LN", "LT", "LX")):
            continue
        df = pd.read_csv(os.path.join(path, file), index_col=0, parse_dates=True)
        df["Return"] = df.iloc[:, 0:1].apply(lambda x: np.log(x.shift(-1) / x))
        df = df.dropna()

        non_positive_LA = df[(df[df.columns[0]] <= 0) | (df['Volatility'] <= 0)]
        df = df.drop(index=non_positive_LA.index)

        df_dict_cc1[file[:-4]] = df
        valid_dates = valid_dates.intersection(df.index)
for name, df in df_dict_cc1.items():
    df_dict_cc1[name] = df[df.index.isin(valid_dates)]

In [21]:
df_dict_cc2 = {}
# Read each file in below folder
path = "../data/investing.com/"
files = os.listdir(path)
for file in files:
    if file.endswith(".csv"):
        # Exclude LB due to data correctness issue
        if file.startswith(("LB", "JN", "LL", "LN", "LT", "LX")):
            continue
        df = pd.read_csv(os.path.join(path, file), index_col=0, parse_dates=True, thousands=",")
        df = df.iloc[:, 0:1].astype(float)
        if (df <= 0).any().any():
            print(f"Non-positive values found in {file[:-4]}")
        df = df[df > 0]
        df_dict_cc2[file[:-4]] = df